## SGDClassifier

In [70]:
sys.path

['',
 '/opt/conda/lib/python36.zip',
 '/opt/conda/lib/python3.6',
 '/opt/conda/lib/python3.6/lib-dynload',
 '/opt/conda/lib/python3.6/site-packages',
 '/opt/conda/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg',
 '/opt/conda/lib/python3.6/site-packages/IPython/extensions',
 '/home/jovyan/.ipython',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/',
 '../src/']

In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score, log_loss
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler, scale
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA
import sys
sys.path.append("../src/")
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from pandas import DataFrame, Series
from sklearn.metrics import mean_squared_error, r2_score
from numpy import sqrt, mean

##### Read in data and one-hot-encode int dtype variables

In [72]:
# Load data
df = pd.read_csv("../data/Series3_6.15.17_padel.csv", index_col=0)
# Eliminate features without variance
df = df.loc[:, (df.std() > 0).values]
# Seperate Series 3 test when IC50 is null
test_index = df.IC50.isnull()
test_df = df.loc[test_index]
df = df.loc[~test_index]
# Remove columns with missing data
df = df.dropna(axis=1)
# Transform discrete with one-hot-encoding
int_cols = df.columns[df.dtypes == 'int64']
float_cols = df.columns[df.dtypes == 'float64']
one_hot_df = pd.get_dummies(df[int_cols].astype('O'))
df = pd.merge(df[float_cols], one_hot_df, left_index=True, right_index=True)
# Split x, y
y_data = df.pop("IC50")
x_data = df.copy()
# Ensure no (+/-) inf or nan due to improper transformation
x_data.replace([np.inf, -np.inf], np.nan, inplace=True)
assert not sum(x_data.isna().sum()), "Unexpected nulls found"

##### Perform non-linear transformations to Series 3

In [73]:
# Perform feature engineering on float columns
for feat in x_data.columns[x_data.dtypes == 'float64']:
    feature_df = x_data.loc[:, feat]
    if feature_df.min() > 0:  # Avoid 0 or negative
        x_data.loc[:, feat + "_log"] = feature_df.apply(np.log)  # log
        x_data.loc[:, feat + "_log2"] = feature_df.apply(np.log2)  # log2
        x_data.loc[:, feat + "_log10"] = feature_df.apply(np.log10)  # log10
        x_data.loc[:, feat + "_cubert"] = feature_df.apply(
            lambda x: np.power(x, 1 / 3))  # cube root
        x_data.loc[:, feat + "_sqrt"] = feature_df.apply(np.sqrt)  # square root
    # Avoid extremely large values, keep around 1M max
    if feature_df.max() < 13:
        x_data.loc[:, feat + "_exp"] = feature_df.apply(np.exp)  # exp
    if feature_df.max() < 20:
        x_data.loc[:, feat + "_exp2"] = feature_df.apply(np.exp2)  # exp2
    if feature_df.max() < 100:
        x_data.loc[:, feat + "_cube"] = feature_df.apply(
            lambda x: np.power(x, 3))  # cube
    if feature_df.max() < 1000:
        x_data.loc[:, feat + "_sq"] = feature_df.apply(np.square)  # square

##### Normalize data

In [74]:

x_data = scale(x_data, with_mean=True)
x_scaler = StandardScaler()
y_scaler = StandardScaler()



In [75]:
reduce_dim = PCA(n_components=350)
pca_result = reduce_dim.fit_transform(x_data)

In [76]:
x_data_pca = pd.DataFrame(data = pca_result)


##### Perform Grid-search to SelecyKBest PCA components and tune SGDClassifier

In [77]:
def variance_scorer(x, y):
    """    
    Get the variance for each column of X.
    
    Because principal components have decreasing variance
    (i.e. PC4 has less variance than PC3 which has less variance
    than PC2 etc.), we can use this function in SelectKBest to select
    only the top X number of principal components.
    
    """
    scores = [np.var(column) for column in x.T]
    return scores, np.array([np.NaN]*len(scores))

##### Validate Model Performance

In [78]:
def score_regressor(x_data, y_data, model, add_train_data=None, verbose=1, pos_split=10):
    """
    Model validation for producing comparable model evaluation. Uses Stratified K-Fold LOOCV adapted
    for regression with the positive equivalent <10 IC50, producing 5 folds.
    :param x_data: Pandas DataFrame object, Series 3 with 47 examples
    :param y_data: Pandas DataFrame or Series object, float datatype, target variables for Series 3
    :param model: must have fit and predict method, use sklearn or wrapper
    :param add_train_data: Additional data to be evenly spread across train splits
    :param verbose: If 0, return dictionary only, if 1 printed results
    :param pos_split: cutoff for positive class in StratifiedKFold (y<pos_split)
    :return: dictionary
    """
    assert isinstance(x_data, DataFrame), "x_data must be a pandas DataFrame"
    assert isinstance(y_data, DataFrame) or isinstance(y_data, Series), "y_data must be pandas DataFrame or Series"
    assert y_data.dtypes == "float", "Expected y_data to be float dtype and received {}".format(y_data.dtypes)

    if add_train_data is not None:
        raise NotImplementedError

    # create logging dictionary to track scores
    scoring_dict = {"r2_score": [], "rmse": []}
    # create y_class series for Stratified K-Fold split at pos_split
    y_class = Series(data=[int(y < pos_split) for y in y_data])
    # num_splits count number of positive examples
    num_splits = sum(y_class.values)
    scoring_dict["num_splits"] = num_splits
    # create splits using stratified kfold
    rskf = RepeatedStratifiedKFold(n_splits=num_splits, n_repeats=10, random_state=36851234)
    # loop through splits
    for train, test in rskf.split(x_data, y_class):
        x_train, x_test = x_data.iloc[train, :], x_data.iloc[test, :]
        y_train, y_test = y_data.iloc[train], y_data.iloc[test]
        # train model, test model with all scoring parameters
        model.fit(x_train, y_train)
        y_ = model.predict(x_test)
        # append scores to logging dictionary
        scoring_dict["r2_score"].append(r2_score(y_test, y_))
        scoring_dict["rmse"].append(sqrt(mean_squared_error(y_test, y_)))
    if verbose == 1:
        # Print contents of dictionary except confusion matrix
        print("with {} splits and {} repeats".format(num_splits, 10))
        for metric in scoring_dict:
            if metric == "num_splits":
                continue
            else:
                print("average {}: {}".format(metric, mean(scoring_dict[metric])))
    return scoring_dict


In [79]:
results = score_regressor(x_data= x_data_pca, y_data = y_data, model = grid.best_estimator_)
                                   

with 5 splits and 10 repeats
average r2_score: -0.20301890116385365
average rmse: 29.567772540388663
